In [ ]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "nc24-cluster" # replace with your own gpu cluster
compute_target = ws.compute_targets[cluster_name]
compute_target

## TensorFlow 2.x - distributed training with tf.distribute
This example is specifically using the TensorFlow MultiWorkerMirroredStrategy (MWM)

In [ ]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='tf2-mwm')

In [ ]:
from azureml.core import Environment

tf_env = Environment.get(ws, name='AzureML-TensorFlow-2.2-GPU')

In [ ]:
import os
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import TensorflowConfiguration

distr_config = TensorflowConfiguration()
distr_config.worker_count = 2
distr_config.parameter_server_count = 0

model_path = os.path.join('outputs', 'keras-model')

src = ScriptRunConfig(source_directory='.',
                      script='keras_mnist_multi_worker.py',
                      arguments=['--epochs', 10, '--model-dir', model_path],
                      compute_target=compute_target,
                      environment=tf_env,
                      distributed_job_config=distr_config)

src.run_config.node_count = 2

**TO DO:**

Once `azureml-sdk` version 1.15.0 is released (ETA 10/5/2020), update the above code to the following:

```python
import os
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import TensorflowConfiguration

distr_config = TensorflowConfiguration(worker_count=2)

model_path = os.path.join('outputs', 'keras-model')

src = ScriptRunConfig(source_directory='.',
                      script='keras_mnist_multi_worker.py',
                      arguments=['--epochs', 10, '--model-dir', model_path],
                      compute_target=compute_target,
                      environment=tf_env,
                      distributed_job_config=distr_config)
```

In [ ]:
run = exp.submit(src)

In [ ]:
run.wait_for_completion(show_output=True)